**Exemplo 02: Previsão de feedbacks de produtos B2W**

Você recebeu um convite para uma consultoria, na qual deve desenvolver um modelo de previsões de feedbacks de clientes em produtos comprados na loja, que serão coletados do instagram.

Os dados que você vai utilizar estão localizados em:
https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv

Na coluna 'review_title' você vai encontrar feedbacks passados dos nossos clientes em nossos produtos e, na coluna 'overall_rating', a nota que foi dada. Esse é o único dado que temos para auxiliar na criação desse modelo de previsões.

Dúvidas? Fale comigo!


**Importando bibliotecas**

In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore

**Obtendo dados**

In [9]:
# Python
try:
    print('Obtendo dados...')

    # constante dos dados
    ENDERECO_DADOS = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/refs/heads/main/B2W-Reviews01.csv'

    # obtendo dados
    df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_state'=='RJ','overall_rating']]

    # excluindo dados não existentes (NaN)
    df = df.dropna(subset=['review_title' ,'overall_rating'])

    # Tranformando colunas em arrays
    texts = np.array(df['review_title'])
    rating = np.array(df['overall_rating'])

    # exibir o total de valores por categoria
    print(df['overall_rating'].value_counts())

    print(df.head())
except Exception as e:
    print('Erro ao obter dados: ', e)

Obtendo dados...
Erro ao obter dados:  '[False] not in index'


C:\Users\36131872024.1\AppData\Local\Temp\ipykernel_27432\1081234357.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ENDERECO_DADOS, sep=',', encoding='utf-8')[['review_state'=='RJ','overall_rating']]


**Vetorização**

In [11]:
# Biblioteca para trabalhar com redes neurais artificiais
# Tensorflow - https://www.tensorflow.org/?hl=pt-br
# Tokenizar
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
# ajustar o tamanho do vetor
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore

try:
    print('Vetorizando texto...')

    # Passo 1: tokenizar
    tokenizer = Tokenizer()

    # Passo 2: Criar o dicionário
    # fit_on_texts: Cria o vocabulário, através do dicionário
    # associando cada token a um índice numérico
    # lembrando que se a palavra aparecer mais de uma vez, ela vai receber o mesmo índice numérico
    tokenizer.fit_on_texts(texts)

    # Passo 3: Vetorizar, ou seja, transformar os tokens em números,
    # a partir do dicionário criado no passo 2
    vetores = tokenizer.texts_to_sequences(texts)
    
    # Passo4: Padronização do tamanho do vetor - pad
    padded_vetores = pad_sequences(vetores)

    print(padded_vetores)

    print('Textos vetorizados!')
    
except Exception as e:
    print('Erro ao vetorizar textos: ', e)

Vetorizando texto...
[[   0    0    0 ...    0    0    3]
 [   0    0    0 ... 2620   30   16]
 [   0    0    0 ...  349   45  155]
 ...
 [   0    0    0 ...    0    9    1]
 [   0    0    0 ...    4   19    3]
 [   0    0    0 ...    1    4   51]]
Textos vetorizados!


**Rede Neural**

In [12]:
# Definição do modelo de rede neural utilizada
from tensorflow.keras.models import Sequential # type: ignore
# Camadas da rede neural
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout # type: ignore
# Otimizador de taxa de aprendizado
from tensorflow.keras.optimizers import Adam # type: ignore

try:
    print('Construindo a rede neural...')

    # Constantes do modelo

    # 1ª Constante: Tamanho do vocabulário
    VOCAB_SIZE = len(tokenizer.word_index) + 1

    # 2ª Constante: Tamanho máximo da sequência
    # É o comprimento máximo de um texto
    MAX_SEQUENCE_LENGHT = padded_vetores.shape[1]

    # 3ª Constante: Tamanho do vetor de entrada
    # A literatura recomenda que inicia-se por uma quantidade 
    # igual a raiz quadrada do tamanho do vocabulário
    # Se o volume de dados for de larga escala, pode-se testar iniciando em um tamanho maior
    # Se o volume de dados for muito pequeno, pode-se testar inciando com um tamanho menor
    # Cuidado com o overfitting, que é quando o modelo "aprende demais" e, começa a 
    # perder a capacidade de generalizar melhor, ou seja, obsevar todas as diferenças textuais
    # Overfitting pode ser observado no treino da rede neural
    VETOR_LENGHT = int(np.sqrt(VOCAB_SIZE))

    # Inicia-se a construção da rede neural
    # Sequential é um fluxo linear de camandas (conforme visto na Aula02_RNA.pptx)
    # São processadas em ordem
    model = Sequential()

    # Camada de entrada
    # Embeddings, na qual os vetores de texto são inseridos
    model.add(
        Embedding(
            input_dim=VOCAB_SIZE,
            output_dim=VETOR_LENGHT,
            input_length=MAX_SEQUENCE_LENGHT))

    # Camada oculta ou intermediária
    # LSTM - Long short-term memory, em português "memória de curto e longo prazo"
    # É onde a magia acontece, ou seja, onde o modelo treina baseado nos seus vetores
    # Números de unidades de memória, que basicamente é a qtde de "neurônios"
    # Primeiro TESTE, EXPERIMENTE somente com 1 camada! Cuidado! com o ovberfitting!
    # Se for necessário, adicionar mais camadas, basta repetir o comando abaixo
    
    # primeirca camada oculta
    model.add(LSTM(128))

    # Se necessário, adicionar outra camada oculta
    # model.add(LSTM(64))

    # Camada de saída - Camada Densa
    # Classificação, que é o caso desse exemplo.
    # Precisa ajustar para a quantidade de classes/categorias
    # 1, 2, 3, 4, 5
    # Em RNA de Classidificação (classes/categorias)
    # A função de ativação mais utilizada é a softmax
    # Função de ativação é um cálculo matemático 
    # que vai determinar a saída de cada neurônio
    # Softmax é uma função (mais utilizada em classificação) 
    # que transforma os valores de saída
    # em PROBABILIDADES que vão de 0 a 1
    # A soma das classes, no nosso caso 5 classes, será igual 1
    # A outra função mais utilizada ReLu (que tb pode ser utilizada em classificação)
    # É uma função que retorna 0, para saídas negativas e o valor original, para
    # saidas maiores que 0 (O que faz ela ser mais indicada para Regressão)
    # model.add(Dense(1, activation='relu'))
    model.add(Dense(5, activation='softmax'))

    # Dropout é um técnica de regularização do resultado, pra minimizar o overfitting
    # Ele desabilita neurônios aleatoriamente, justamente, para tentar minimizar overfitting
    # Cuidado com o underfitting, se reduzir demais, vai dar ruim!
    model.add(Dropout(0.05)) # desativar 5% dos neurônios, aleatoriamente

    # construir o modelo
    # É literamente pegar as definições anteriores e construir o modelo
    # input_shape: é o formato do dados de entrada e ainda o tamanho máximo do texto (MAX_SEQUENCE_LENGHT)
    model.build(input_shape=(None,MAX_SEQUENCE_LENGHT))
    
    # Otimizador de taxa de aprendizado
    # importante para ajustar, em casos de overfitting
    # Adam: É nosso otimizador que vaia ajustar essa taxa de aprendizado
    # learning_rate: Quanto menor, melhor o aprendizado. Menos risco de overfitting
    otimizador = Adam(learning_rate=0.0001)

    # compilar o modelo
    # Verificar se possui algum erro ou se tá de boa
    # Além disso, vamos informar o otimizador e a nossa métrica de perda (loss)
    model.compile(optimizer=otimizador, loss='sparse_categorical_crossentropy')

    #model.summary()
    print('Modelo configurado e criado')

except Exception as e:
    print('Erro ao construir a rede neural: ', e)

Construindo a rede neural...
Modelo configurado e criado


C:\Users\36131872024.1\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
from sklearn.model_selection import train_test_split#type:ignore

try:
    print('Treinar o modelo de rede neural')

    X_train, X_test, y_train, y_test = train_test_split(
        padded_vetores,
        rating,
        test_size=0.2,
        random_state=42
    )

    # Na Classificação
    # Qdo a gente constroi um RNA de classificação
    # os rótulo que a RNA atribui [0,1,2,3,4] e os meus dados são [1,2,3,4,5]
    y_train_adjusted = y_train - 1
    y_test_adjusted = y_test - 1

    # Aplicar pesos as categorias
    pesos = len(df['overall_rating'])/df['overall_rating'].value_counts()

    print(pesos)

    # o treino da rede neural
    model.fit(
        X_train, # x treino
        y_train_adjusted, # y treino
        epochs=5, # Épocas de estudo da rede neural. Qto mais epochs, mais aprendizado, 
        # mais processamento, porém cuidado com o overfitting. A regre aqui é experimentação
        # avaliando a peda de treino, com a perda de teste
        batch_size=32, # Qto maior tamanho da batch, tende a um menor o aprendizado, porém evita overfitting
        # Qto menor o tamanho da batch, tende a um maior aprendizado
        class_weight = pesos.to_dict(), # precisa receber os pesos em uma estrutura de dados, do tipo dicionário
        validation_data=(X_test, y_test_adjusted)
    )

except Exception as e:
    print('Erro ao treinar a rede neural: ', e)

Treinar o modelo de rede neural
overall_rating
5     2.759758
4     4.090785
1     4.841312
3     8.112968
2    15.777207
Name: count, dtype: float64
Epoch 1/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 58s 17ms/step - loss: 11.3596 - val_loss: 1.1988
Epoch 2/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 58s 18ms/step - loss: 9.9046 - val_loss: 1.2185
Epoch 3/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 60s 18ms/step - loss: 9.6073 - val_loss: 1.1101
Epoch 4/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 59s 18ms/step - loss: 9.8057 - val_loss: 1.1291
Epoch 5/5
3302/3302 ━━━━━━━━━━━━━━━━━━━━ 60s 18ms/step - loss: 9.5248 - val_loss: 1.1002


In [14]:
# textos para realizar a previsão, baseada no modelo de rede neural
# Pode e deve conectar em um fonte de dados, para realizar as previsões de sentimentos
novos_textos = [
    "Muito bom, gostei bastante. Top demais! Compensa muito!",
    "Não recomendo, péssimo produto. Não funciona"
]

# Vetorizar
novas_sequencias = tokenizer.texts_to_sequences(novos_textos)
novas_sequencias_padded = pad_sequences(novas_sequencias)

# análise preditiva
predicoes = model.predict(novas_sequencias_padded)

# Formatar valores de saída
np.set_printoptions(suppress=True, precision=4)

print("Previsões: ", predicoes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Previsões:  [[0.0001 0.0002 0.0056 0.2863 0.7078]
 [0.8209 0.0881 0.0372 0.0237 0.03  ]]
